# Dataset generation step by step

In [1]:
!pip install Pillow

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

from livelossplot import PlotLosses
from pycm import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
import numpy as np
import random
from PIL import Image
from tqdm import tqdm
from math import ceil

Image.MAX_IMAGE_PIXELS = None
import cv2
from matplotlib.patches import Ellipse,Rectangle

import pylab as plt
import numpy as np
import math
import pandas as pd
import os


In [3]:
df = pd.read_csv("Catalog_Mars_Release_2020_1kmPlus_FullMorphData.csv",engine='python',delimiter=',')
df

,CRATER_ID,LAT_CIRC_IMG,LON_CIRC_IMG,LAT_ELLI_IMG,LON_ELLI_IMG,DIAM_CIRC_IMG,DIAM_CIRC_SD_IMG,DIAM_ELLI_MAJOR_IMG,DIAM_ELLI_MINOR_IMG,DIAM_ELLI_ECCEN_IMG,...,INT_MORPH1,INT_MORPH2,INT_MORPH3,CONF,NOTES,DEG_RIM,DEG_EJC,DEG_FLR,DEG_LEV_1,DEG_LEV
0,01-1-000003,79.257599,211.909949,79.257584,211.910021,19.526698,0.230187,19.746718,19.286041,0.214742,...,CpxUnc,NaN,Floor Deposits,NaN,NaN,1.0,1.0,1.0,3,3
1,01-1-000004,78.696512,207.162344,78.696517,207.162105,4.232291,0.089167,4.449180,4.019798,0.428605,...,NaN,NaN,Floor Deposits,NaN,NaN,3.0,2.0,1.0,6,6
2,01-1-000012,77.791422,186.586023,77.791443,186.585690,8.542042,0.193030,8.818632,8.251528,0.352817,...,CpxFF,NaN,NaN,NaN,NaN,2.0,2.0,2.0,6,6
3,01-1-000013,76.655040,194.807812,76.655020,194.807973,7.967843,0.139290,8.268589,7.662693,0.375745,...,CpxFF,NaN,NaN,NaN,NaN,3.0,2.0,3.0,8,8
4,01-1-000014,76.976285,195.615795,76.976294,195.616254,20.948878,0.340399,21.550893,20.326902,0.332212,...,CpxFF,NaN,Floor Deposits,NaN,NaN,3.0,2.0,3.0,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392426,16-4-007160,-51.253765,93.980015,-51.253710,93.980005,1.325088,0.016568,1.365264,1.279374,0.349092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,-1
392427,16-4-007161,-52.010160,131.870216,-52.010131,131.870172,1.266052,0.014247,1.289768,1.238597,0.278881,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,-1
392428,16-4-007162,-55.929621,165.240464,-55.929621,165.240425,1.093010,0.033980,1.144802,1.035258,0.426872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,-1
392429,16-4-007163,-57.577402,150.020821,-57.577196,150.021399,1.765619,0.029925,1.866267,1.708802,0.402033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,-1


## Generate the dataset by matching lat long 

In [4]:
def generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,target_size,im_name,name):
    
    if os.path.exists(pathsave+'/labels') == False:
        os.mkdir(pathsave+'/labels')
    if os.path.exists(pathsave+'/images') == False:
        os.mkdir(pathsave+'/images')
    
    images_path = pathsave+'/images'
    labels_path = pathsave+'/labels'
    df1 = df[(df['LAT_CIRC_IMG']>min_lat) & (df['LAT_CIRC_IMG']<max_lat)]
    df1 = df1[df1['LON_CIRC_IMG']>min_long]
    df1 = df1[df1['LON_CIRC_IMG']<max_long]

    df2 = df1.copy()
    df2['x'] = 1
    df2['y'] = 1
    df2['w'] = 1
    df2['h'] = 1
    df2['r']  = 1

    x = df1['LON_ELLI_IMG']
    y = df1['LAT_ELLI_IMG']
    r = df1['DIAM_ELLI_MAJOR_IMG']/2
    lon_ratio = 3378.4*np.cos(math.pi/180*y)*math.pi/180
    lat_ratio = np.pi*3389.5/180
    df2['x'] = x - r/lon_ratio
    df2['y'] = y - r/lat_ratio
    df2['w'] = 2*r/lon_ratio
    df2['h'] = 2*r/lat_ratio
    df2['r'] = r

    img = plt.imread(im_name)
    x = img.shape[1]
    y = img.shape[0]
    df3 = df2.copy()

    df3['x'] = x / (max_long - min_long) * (df2['x'] - min_long)
    df3['y'] = y - y / (max_lat-min_lat) * (df2['y'] - min_lat)
    df3['w'] = x / (max_long-min_long) * df2['w']
    df3['h'] = y / (max_lat-min_lat) * df2['h']


    h, w = img.shape
    c=0
    # x - h, y - w
    crop_size = target_size
    num_y = int(math.floor(h / target_size))
    num_x = int(math.floor(w / target_size))
    print(f"width: {w}, height: {h}, crop size: {crop_size}")
    print(f"num in x direction: {num_x}, num in y direction: {num_y}")


    clean_images = f'del {images_path}'
    clean_labels = f'del {labels_path}'
    !{clean_images}
    !{clean_labels}

    # Calculate initial number of crops horizontally and vertically without overlap
    num_x = ceil(w / crop_size)
    num_y = ceil(h / crop_size)

    # Dynamically calculate overlap based on remaining space for the last crop
    overlap_x = (num_x * crop_size - w) / (num_x - 1) if num_x > 1 else 0
    overlap_y = (num_y * crop_size - h) / (num_y - 1) if num_y > 1 else 0

    for i in tqdm(range(num_y)):
        for j in range(num_x):
            x_low = j * crop_size - j * overlap_x
            y_low = i * crop_size - i * overlap_y

            # Ensure the calculated starting points are within the image boundaries
            x_low, y_low = math.floor(max(0, x_low)), math.floor(max(0, y_low))
            x_high, y_high = x_low + crop_size, y_low + crop_size
            x_high = math.floor(min(x_high, w))
            y_high = math.floor(min(y_high, h))

#             print('crop_size:',x_high-x_low, y_high-y_low)
            
            curr_list = []
            offset = 0
            df5 = df3.copy()
            img_crop = img[y_low:y_high, x_low:x_high]
            base_name_img = images_path+'/'+name+'_'+str(x_low)+'_'+str(y_low)+'_'+str(max_lat)+ '_' + str(min_lat)+ '_' + str(max_long)+ '_' + str(min_long)
            base_name_label = labels_path+'/'+name+'_'+ str(x_low) +'_'+ str(y_low) +'_'+str(max_lat)+ '_' + str(min_lat)+ '_' + str(max_long)+ '_' + str(min_long)
            cv2.imwrite(base_name_img+'.jpg',img_crop)
#             with Image.open(base_name_img+'.jpg') as img:
#                     # Resize the image
#                     resized_img = img.resize(1024, Image.ANTIALIAS)
#                     resized_img.save(output_path)
                            
            df5 = df3[(df3['y']>=y_low-offset) & (df3['y']<=y_high+offset)]
            df5 = df5[(df5['x']>=x_low-offset) & (df5['x']<=x_high+offset)]



            if len(df5) == 0:
    #             fileObject = open(''+str(c)+'.txt','a')
                pass
            else:
                x_crop = img_crop.shape[1]
                y_crop = img_crop.shape[0]

                for k in range(len(df5)):
                    y = float(df1['LAT_ELLI_IMG'][df1['LAT_ELLI_IMG'].index[i]])
                    lon_ratio = 3378.4*math.cos(math.pi/180*y)*math.pi/180
                    lat_ratio = math.pi*3389.5/180

                    if df5['h'][df5['h'].index[k]]>60:

                        X = (df5['x'][df5['x'].index[k]]-x_low)/x_crop+df5['w'][df5['w'].index[k]]/x_crop/2
                        Y = ((df5['y'][df5['y'].index[k]]-20*df5['r'][df5['r'].index[k]]- y_low))/y_crop+df5['h'][df5['h'].index[k]]/y_crop/2
                        W = df5['w'][df5['w'].index[k]]/x_crop
                        H = df5['h'][df5['h'].index[k]]/y_crop
                        curr_list.append(['0', max(min(X, 0.99), 0.01), max(min(Y, 0.99), 0.01), max(min(W, 0.99), 0.01), max(min(H, 0.99), 0.01)])

                    elif 10<=df5['h'][df5['h'].index[k]]<=60:
                        X = (df5['x'][df5['x'].index[k]]-df5['r'][df5['r'].index[k]]*2-x_low)/x_crop+df5['w'][df5['w'].index[k]]/x_crop/2
                        Y = ((df5['y'][df5['y'].index[k]]-20*df5['r'][df5['r'].index[k]]- y_low))/y_crop+df5['h'][df5['h'].index[k]]/y_crop/2
                        W = df5['w'][df5['w'].index[k]]/x_crop
                        H = df5['h'][df5['h'].index[k]]/y_crop
                        curr_list.append(['0', max(min(X, 0.99), 0.01), max(min(Y, 0.99), 0.01), max(min(W, 0.99), 0.01), max(min(H, 0.99), 0.01)])

                    elif 10<=df5['h'][df5['h'].index[k]]<=60:
                        X = (df5['x'][df5['x'].index[k]]-df5['r'][df5['r'].index[k]]*2-x_low)/x_crop+df5['w'][df5['w'].index[k]]/x_crop/2
                        Y = ((df5['y'][df5['y'].index[k]]-20*df5['r'][df5['r'].index[k]]- y_low))/y_crop+df5['h'][df5['h'].index[k]]/y_crop/2
                        W = df5['w'][df5['w'].index[k]]/x_crop
                        H = df5['h'][df5['h'].index[k]]/y_crop
                        curr_list.append(['0', max(min(X, 0.99), 0.01), max(min(Y, 0.99), 0.01), max(min(W, 0.99), 0.01), max(min(H, 0.99), 0.01)])

                    else:
                        pass

                with open(base_name_label +'.txt','w') as fileObject:
                    for values in curr_list:
                        for ip in values:
                            fileObject.write(str(ip))
                            fileObject.write(' ')
                        fileObject.write('\n')

            c+=1
            


## Generate the yolo dataset by matching the lat long with robbins dataset

The following code will generate folders with images and labels for each THEMIS image tile. It involves overlapping at the image boundaries to ensure full coverage. You can set your own preference for the train-test-validation split.


In [ ]:
# set the size of the image tiles 
size = 416

#### MareTyrrhenum

In [6]:
from math import ceil
from tqdm import tqdm
name = 'MareTyrrhenum'
# pathsave = 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name

In [7]:
if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_MareTyrrhenum_30S090E_100mpp.jpg'
output_path = 'dataset_generation_old/detection_img/'+str(size) # replace with your output folder path

max_lat = 0
min_lat = -30
max_long = 135
min_long = 90
# generate_data(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26675, height: 17783, crop size: 416
num in x direction: 64, num in y direction: 42


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 43/43 [00:34<00:00,  1.23it/s]


#### Tharsis

In [44]:
name = 'Tharsis'
# pathsave = 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name
if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)

In [45]:
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_Tharsis_000N225E_100mpp.tif'
min_lat = 0
max_lat = 30
min_long = 225
max_long = 270
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26675, height: 17783, crop size: 6144
num in x direction: 4, num in y direction: 2


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 3/3 [00:13<00:00,  4.66s/it]


#### SyrtisMajor

In [14]:
name = 'Syrtis'
# pathsave = 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name
if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)
    

In [15]:
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_SyrtisMajor_00N45E_100mpp.tif'
min_lat = 0
max_lat = 30
min_long = 45
max_long = 90
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26674, height: 17783, crop size: 416
num in x direction: 64, num in y direction: 42


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 43/43 [00:32<00:00,  1.31it/s]


#### SinusSabaeus

In [44]:
name = 'SinusSabaeus'
# pathsave = 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name
if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)
    

In [45]:
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_SinusSabaeus_30S00E_100mpp.tif'
min_lat = -30
max_lat = 0
min_long = 0
max_long = 45
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26674, height: 17783, crop size: 416
num in x direction: 64, num in y direction: 42


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 43/43 [00:39<00:00,  1.08it/s]


#### PhoenicisLacus

In [26]:
# size = 3072
name = 'PhoenicisLacus'
# pathsave = 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name
if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)
    

In [27]:
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_PhoenicisLacus_30S225E_100mpp.tif'
min_lat = -30
max_lat = 0
min_long = 225
max_long = 270
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26675, height: 17783, crop size: 416
num in x direction: 64, num in y direction: 42


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 43/43 [00:22<00:00,  1.90it/s]


#### Memnonia

In [37]:
name = 'menomia'
# pathsave = 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name

if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)
    

In [38]:
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_Memnonia_30S180E_100mpp.tif'
min_lat = -30
max_lat = 0
min_long = 180
max_long = 225
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26674, height: 17783, crop size: 1024
num in x direction: 26, num in y direction: 17


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 18/18 [00:24<00:00,  1.34s/it]


#### MargaritiferSinus

In [53]:
name = 'MargaritiferSinus'
# pathsave = 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name

if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)
    

In [54]:
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_MargaritiferSinus_30S315E_100mpp.jpg'
min_lat = -30
max_lat = 0
min_long = -45
max_long = 0
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26674, height: 17783, crop size: 416
num in x direction: 64, num in y direction: 42


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 43/43 [00:19<00:00,  2.23it/s]


#### LunaePalus

In [56]:
name = 'LunaePalus'
# pathsave = 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name
if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)
    

In [57]:
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_LunaePalus_00N270E_100mpp.jpg'
min_lat = 0
max_lat = 30
min_long = 270
max_long = 315
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26674, height: 17783, crop size: 416
num in x direction: 64, num in y direction: 42


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 43/43 [00:34<00:00,  1.26it/s]


#### Iapygia

In [59]:
name = 'Iapygia'
# pathsave = 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name
if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)
    

In [60]:
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_Iapygia_30S45E_100mpp.tif'
min_lat = -30
max_lat = 0
min_long = 45
max_long = 90
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26674, height: 17783, crop size: 416
num in x direction: 64, num in y direction: 42


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 43/43 [00:52<00:00,  1.23s/it]


#### Coprates

In [62]:
name = 'Coprates'
# pathsave = 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name
if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)
    

In [63]:
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_Coprates_30S270E_100mpp.tif'
min_lat = -30
max_lat = 0
min_long = 270
max_long = 315
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26674, height: 17783, crop size: 416
num in x direction: 64, num in y direction: 42


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 43/43 [00:36<00:00,  1.19it/s]


#### Elysium

In [65]:
name = 'Elysium'
# pathsave = 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name
if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)
    

In [66]:
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_Elysium_00N135E_100mpp.tif'
min_lat = 0
max_lat = 30
min_long = 135
max_long = 180
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26674, height: 17783, crop size: 416
num in x direction: 64, num in y direction: 42


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 43/43 [00:27<00:00,  1.55it/s]


#### Arabia

In [68]:
name = 'Arabia'
# pathsave = 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name
if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)
    

In [69]:
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_Arabia_000N000E_100mpp.tif'
min_lat = 0
max_lat = 30
min_long = 0
max_long = 45
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26674, height: 17783, crop size: 416
num in x direction: 64, num in y direction: 42


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 43/43 [00:49<00:00,  1.14s/it]


#### Amazonis

In [71]:
name = 'Amazonis'
# pathsave = 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name
if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)
    

In [72]:
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_Amazonis_00N180E_100mpp.tif'
min_lat = 0
max_lat = 30
min_long = 180
max_long = 225
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26674, height: 17783, crop size: 416
num in x direction: 64, num in y direction: 42


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 43/43 [00:19<00:00,  2.26it/s]


#### Amenthes

In [74]:
name = 'Amenthes'
# pathsave = 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name
if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)
    

In [75]:
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_Amenthes_000N090E_100mpp.tif'
min_lat = 0
max_lat = 30
min_long = 90
max_long = 135
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26675, height: 17783, crop size: 416
num in x direction: 64, num in y direction: 42


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 43/43 [00:29<00:00,  1.44it/s]


#### Aeolis

In [77]:
name = 'Aeolis'
# pathsave =| 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name
if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)
    

In [78]:
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_Aeolis_30S135E_100mpp.tif'
min_lat = -30
max_lat = 0
min_long = 135
max_long = 180
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26674, height: 17783, crop size: 416
num in x direction: 64, num in y direction: 42


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 43/43 [00:35<00:00,  1.23it/s]


#### OxiaPalus

In [80]:
name = 'OxiaPalus'
# pathsave = 'dataset_generation_old/dataset_'+str(size)+'_'+name
pathsave = 'dataset_generation_old/size_variant/dataset_'+str(size)+'_'+name
if os.path.exists(pathsave) == False:
    os.mkdir(pathsave)
    

In [81]:
im_name = 'Themis/THEMIS_DayIR_ControlledMosaic_OxiaPalus_00N315E_100mpp.tif'
min_lat = 0
max_lat = 30
min_long = -45
max_long = 0
generate_data_overlap(max_lat,min_lat,max_long,min_long,pathsave,df,size,im_name,name)

width: 26674, height: 17783, crop size: 416
num in x direction: 64, num in y direction: 42


Parameter format not correct - "size_variant".
Parameter format not correct - "size_variant".
100%|██████████| 43/43 [00:15<00:00,  2.86it/s]
